In [114]:
library('transformeR')
library(raster)

In [349]:
#archivosdf <- list.files(path = '../3.Datos generados/dataframes', pattern = "^df_grid_", full.names = TRUE)
#archivosdf
#archivos_grid <- list.files(path = '../1.Upscaling/upscaling_ba', full.names = TRUE)
archivos_raster <- list.files(path = '../3.Datos generados/raster', full.names = TRUE)
#archivos_raster

In [350]:
df_grid_25_sinInterpolar <- get(load('../3.Datos generados/dataframes/df_grid_25.Rdata'))

Con vecinos más cercanos

In [351]:
lista_variables <- c('grid_05','grid_1','grid_15','grid_2','grid_25','grid_3', 'grid_4','grid_5')
lista_df_interpoladas <- list()
for (i in 1:length(archivos_raster)){
    lista_interpoladas <- list()
    for (j in seq(1,9)){
        grid_raster <- raster(archivos_raster[i], band = j)  
        #creamos un raster vacío a 2.5
        grid_raster_2.5 <- raster(extent(grid_raster), res = 2.5)
        #realizamos interpolación por vecinos cercanos (ngb)
        raster_interp <- resample(grid_raster, grid_raster_2.5, method = "ngb")
        vector_raster_interp <- as.vector(raster_interp)
        columna = paste0('layer_',j)
        lista_interpoladas[[columna]] <- vector_raster_interp 
    }
    coordenadas_interpolada <- coordinates(raster_interp)
    df_interpoladas <- data.frame(lista_interpoladas)
    df_interpoladas <- cbind(coordenadas_interpolada, df_interpoladas)
    colnames(df_interpoladas) <- c('X','Y','FsOrNot','MainStart','MainEnd','SencondStart','SencondEnd','Length','C','P','FBA')
    var_name = paste0('df_interpoladas_',lista_variables[i])
    lista_df_interpoladas[[var_name]] <- df_interpoladas
}

In [352]:
vector_grids <- c('grid_05','grid_1','grid_15','grid_2','grid_25','grid_3', 'grid_4','grid_5')
variablesComparar <- c('FsOrNot','MainStart','MainEnd','SencondStart','SencondEnd','Length','C','P','FBA')
lista_grids_MAES <- list()
for (i in 1:length(lista_df_interpoladas)){
    grid <- vector_grids[i]
    interpolada <- lista_df_interpoladas[[i]]
    MAE_s <- c()
    for (var in variablesComparar){
        var_inter <- unlist(interpolada[var])
        names(var_inter) <- NULL
        var_grid_25_sinInterpolar <-unlist(lista_df_interpoladas$'df_interpoladas_grid_25'[var])
        names(var_grid_25_sinInterpolar) <- NULL
        diff_var <- c()
        for (item in 1:length(var_inter)){
            if (is.na(var_inter[item]) & is.na(var_grid_25_sinInterpolar[item])){
                diff_var <- c(diff_var ,0)
            }else{
                diff_var <- c(diff_var, abs(var_inter[item] - var_grid_25_sinInterpolar[item]) )
            }
        }
    diff_var <- na.omit(diff_var)    
    MAE_var = mean(diff_var)
    MAE_s <- c(MAE_s, MAE_var)
    }
    names(MAE_s) <- variablesComparar
    lista_grids_MAES[[vector_grids[i]]] <- MAE_s
}

In [353]:
t(data.frame(lista_grids_MAES))

,FsOrNot,MainStart,MainEnd,SencondStart,SencondEnd,Length,C,P,FBA
grid_05,0.08825231,0.3156956,0.4404479,0.11163793,0.07392241,0.5786762,0.12494611,0.1940091,0.29475797
grid_1,0.05304784,0.2102261,0.3625993,0.04306864,0.02836733,0.3652475,0.06243892,0.1472351,0.02222783
grid_15,0.03288966,0.1345368,0.2945048,0.04104023,0.02895165,0.2678767,0.04377659,0.1122534,0.02021273
grid_2,0.02662037,0.1179152,0.2599088,0.03910671,0.02728375,0.2291914,0.03717047,0.1044816,0.02042865
grid_25,0.00000000,0.0000000,0.0000000,0.00000000,0.00000000,0.0000000,0.00000000,0.0000000,0.00000000
grid_3,0.03182870,0.1411636,0.3136083,0.04522767,0.03188347,0.2865113,0.04608705,0.1303905,0.02731663
grid_4,0.04812886,0.1790455,0.3975073,0.05676404,0.03925834,0.3713649,0.06190737,0.1624733,0.03452541
grid_5,0.06172840,0.1892475,0.4291735,0.05645753,0.03969833,0.3910362,0.07110289,0.1731939,0.04002349


In [354]:
library(xtable)
table <- xtable(t(data.frame(lista_grids_MAES)))
print(table, include.rownames = FALSE)

% latex table generated in R 3.6.3 by xtable 1.8-4 package
% Sat Sep 16 14:41:25 2023
\begin{table}[ht]
\centering
\begin{tabular}{rrrrrrrrr}
  \hline
FsOrNot & MainStart & MainEnd & SencondStart & SencondEnd & Length & C & P & FBA \\ 
  \hline
0.09 & 0.32 & 0.44 & 0.11 & 0.07 & 0.58 & 0.12 & 0.19 & 0.29 \\ 
  0.05 & 0.21 & 0.36 & 0.04 & 0.03 & 0.37 & 0.06 & 0.15 & 0.02 \\ 
  0.03 & 0.13 & 0.29 & 0.04 & 0.03 & 0.27 & 0.04 & 0.11 & 0.02 \\ 
  0.03 & 0.12 & 0.26 & 0.04 & 0.03 & 0.23 & 0.04 & 0.10 & 0.02 \\ 
  0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 \\ 
  0.03 & 0.14 & 0.31 & 0.05 & 0.03 & 0.29 & 0.05 & 0.13 & 0.03 \\ 
  0.05 & 0.18 & 0.40 & 0.06 & 0.04 & 0.37 & 0.06 & 0.16 & 0.03 \\ 
  0.06 & 0.19 & 0.43 & 0.06 & 0.04 & 0.39 & 0.07 & 0.17 & 0.04 \\ 
   \hline
\end{tabular}
\end{table}


Con bilinear

In [344]:
lista_variables <- c('grid_05','grid_1','grid_15','grid_2','grid_25','grid_3', 'grid_4','grid_5')
lista_df_interpoladas <- list()
for (i in 1:length(archivos_raster)){
    lista_interpoladas <- list()
    for (j in seq(1,9)){
        grid_raster <- raster(archivos_raster[i], band = j)  
        #creamos un raster vacío a 2.5
        grid_raster_2.5 <- raster(extent(grid_raster), res = 2.5)
        #realizamos interpolación por vecinos cercanos (ngb)
        raster_interp <- resample(grid_raster, grid_raster_2.5, method = "bilinear")
        vector_raster_interp <- as.vector(raster_interp)
        columna = paste0('layer_',j)
        lista_interpoladas[[columna]] <- vector_raster_interp 
    }
    coordenadas_interpolada <- coordinates(raster_interp)
    df_interpoladas <- data.frame(lista_interpoladas)
    df_interpoladas <- cbind(coordenadas_interpolada, df_interpoladas)
    colnames(df_interpoladas) <- c('X','Y','FsOrNot','MainStart','MainEnd','SencondStart','SencondEnd','Length','C','P','FBA')
    var_name = paste0('df_interpoladas_',lista_variables[i])
    lista_df_interpoladas[[var_name]] <- df_interpoladas
}

In [346]:
vector_grids <- c('grid_05','grid_1','grid_15','grid_2','grid_25','grid_3', 'grid_4','grid_5')
variablesComparar <- c('FsOrNot','MainStart','MainEnd','SencondStart','SencondEnd','Length','C','P','FBA')
lista_grids_MAES <- list()
for (i in 1:length(lista_df_interpoladas)){
    grid <- vector_grids[i]
    interpolada <- lista_df_interpoladas[[i]]
    MAE_s <- c()
    for (var in variablesComparar){
        var_inter <- unlist(interpolada[var])
        names(var_inter) <- NULL
        var_grid_25_sinInterpolar <-unlist(lista_df_interpoladas$'df_interpoladas_grid_25'[var])
        names(var_grid_25_sinInterpolar) <- NULL
        diff_var <- c()
        for (item in 1:length(var_inter)){
            if (is.na(var_inter[item]) & is.na(var_grid_25_sinInterpolar[item])){
                diff_var <- c(diff_var ,0)
            }else{
                diff_var <- c(diff_var, abs(var_inter[item] - var_grid_25_sinInterpolar[item]) )
            }
        }
    diff_var <- na.omit(diff_var)    
    MAE_var = mean(diff_var)
    MAE_s <- c(MAE_s, MAE_var)
    }
    names(MAE_s) <- variablesComparar
    lista_grids_MAES[[vector_grids[i]]] <- MAE_s
}

In [347]:
t(data.frame(lista_grids_MAES))

,FsOrNot,MainStart,MainEnd,SencondStart,SencondEnd,Length,C,P,FBA
grid_05,0.09111032,0.4266898,0.4609441,0.17109992,0.12693339,0.5702300,0.07349209,0.1615045,0.299206427
grid_1,0.05365668,0.2339334,0.3705870,0.08236500,0.05539743,0.3801183,0.05327888,0.1317174,0.011157023
grid_15,0.03680127,0.1702487,0.3340071,0.07689754,0.05477719,0.3036438,0.04154976,0.1119714,0.007999769
grid_2,0.03251742,0.1640000,0.3228464,0.08242287,0.06060365,0.2778054,0.03722217,0.1082916,0.010575980
grid_25,0.00000000,0.0000000,0.0000000,0.00000000,0.00000000,0.0000000,0.00000000,0.0000000,0.000000000
grid_3,0.04201323,0.2012983,0.3876081,0.09580311,0.07239528,0.3347066,0.05007797,0.1315018,0.022683365
grid_4,0.05798298,0.2312029,0.4679415,0.12115645,0.08787251,0.4188026,0.06603080,0.1623885,0.033004167
grid_5,0.07767077,0.2534552,0.4915593,0.12443753,0.08922049,0.4585908,0.08245383,0.1782202,0.044799817


In [ ]:
df_grid_05_ordenado <- data.frame()
for (row1 in 1:nrow(df.series_grid_05)){
    for (row2 in 1:nrow(df_grid_05)){
        seriesx = df.series_grid_05$coord_x[row1]
        seriesy = df.series_grid_05$coord_y[row1]
        gridx = df_grid_05$coord_x[row2]
        gridy = df_grid_05$coord_y[row2]
        if (seriesx == gridx & seriesy == gridy){
            df_grid_05_ordenado <- rbind(df_grid_05_ordenado, df_grid_05[row2,])
        }
    }
}

In [65]:
coords_grid_25 <- expand.grid(getCoordinates(grid_25)$x, getCoordinates(grid_25)$y)
colnames(coords_grid_25) <- c('x','y')
#coords_grid_25 <- list('x'=getCoordinates(grid_25)$x, 'y' = getCoordinates(grid_25)$y)

In [14]:
grid_25 <- get(load('../1.Upscaling/upscaling_ba/ba_grid_25.Rdata'))
grid_5 <- get(load('../1.Upscaling/upscaling_ba/ba_grid_5.Rdata'))
grid_4 <- get(load('../1.Upscaling/upscaling_ba/ba_grid_4.Rdata'))
grid_2 <- get(load('../1.Upscaling/upscaling_ba/ba_grid_2.Rdata'))
grid_1 <- get(load('../1.Upscaling/upscaling_ba/ba_grid_1.Rdata'))

In [52]:
sgdf_5 <- clim2sgdf(climatology(grid_5))
#sgdf_4 <- clim2sgdf(climatology(grid_4))

[2023-09-15 13:37:06] - Computing climatology...

[2023-09-15 13:37:06] - Done.



In [67]:
grid_5_interp <-interpGrid(grid = redim(grid_5), new.coordinates =coords_grid_25)

[2023-09-15 13:44:58] Calculating nearest neighbors...

[2023-09-15 13:51:01] Performing nearest interpolation... may take a while



ERROR: Error in grid$Data[i, , ind.NN.y[k, l], ind.NN.x[k, l]]: incorrect number of dimensions


In [32]:
grid_1_interp <-interpGrid(grid = redim(grid_1), new.coordinates =coords_grid_25)

[2023-09-11 15:15:40] Calculating nearest neighbors...

[2023-09-11 15:22:30] Performing nearest interpolation... may take a while



ERROR: Error in grid$Data[i, , ind.NN.y[k, l], ind.NN.x[k, l]]: incorrect number of dimensions


In [ ]:
grid_5_interp <-interpGrid(grid = grid_5, new.coordinates =coords_grid_25)

In [7]:
getShape(grid_1)

var member   time    lat    lon 
     1      1    256    180    360

de raster --> spatialGridDataFrame
transformeR::sgdf2clim 
spatialGridDf es un tipologia para usarlo con raster 